## Algoritmo Genético

No jupyter anterior utilizamos o algoritmo Gradiente Decrescente para otimizar a função objetivo, afim de comparações, dessa vez utilizaremos um algoritmo genético.

In [1]:
from random import randint, uniform, random
from tqdm import tqdm
import numpy as np

In [2]:
# Manipulação das matrizes

def somar(A, B):
    C = []
    nLinhasA, nLinhasB = len(A), len(B)
    nColA, nColB = len(A[0]), len(B[0])
    
    for i in range (nLinhasA):
        linha = [0]*nColA
        C.append(linha)
        for j in range(nColA):
            C[i][j] = A[i][j] + B[i][j]

    return C

def sub(A, B):
    C = []
    nLinhasA, nLinhasB = len(A), len(B)
    nColA, nColB = len(A[0]), len(B[0])
    
    for i in range (nLinhasA):
        linha = [0]*nColA
        C.append(linha)
        for j in range(nColA):
            C[i][j] = A[i][j] - B[i][j]

    return C


#################################################################

## d (psi) / dt = -i * H * |psi>
def dpsidt(t, psi, H): # A derivada da onda em relação ao tempo não tem dependência temporal
    A = np.zeros((2,2), dtype=np.complex_)
    A = np.dot(complex(0,1),H) # i * H
    return -1*np.matmul(A,psi) # - i * H * |psi>

# Runge-Kutta de quarta ordem

def rungeKutta(onda, fatorRungeKutta, hamiltoniano, tempoFinal = 2, tempoInicial = 0):
    
    resultado = np.zeros((2,2), dtype=np.complex_)
    indice = 0

    for i in range(0, 2):
        
        resultado[0][indice] = onda[0][0]
        resultado[1][indice] = onda[1][0]
        
        k1 = dpsidt(tempoInicial, onda, hamiltoniano)
        k2 = dpsidt(tempoInicial + 0.5 * fatorRungeKutta, somar(onda, np.dot((0.5*fatorRungeKutta), k1)), hamiltoniano)
        k3 = dpsidt(tempoInicial + 0.5 * fatorRungeKutta, somar(onda, np.dot((0.5*fatorRungeKutta), k2)), hamiltoniano)
        k4 = dpsidt(tempoInicial + fatorRungeKutta, somar(onda, np.dot(fatorRungeKutta, k3)), hamiltoniano)
        
        ## y(i+1) = y(i) + h/6*(k1+2*k2+2*k3+k4)
        
        A = somar(np.dot(2,k3), k4)
        B = somar(np.dot(2,k2), k1)
        C = somar(A, B)
 
        onda = somar(onda,np.dot((fatorRungeKutta / 6.0),(C)))
    
        tempoInicial += fatorRungeKutta
        
        indice += 1
        
    runge = np.zeros((2,1), dtype=np.complex_)
    runge[0][0] = resultado[0][indice-1]
    runge[1][0] = resultado[1][indice-1]
    return runge

def funcaoObjetivo(x, iteracao, ondaDestino, hamiltoniano, onda, horizonteAnalisavel, fatorRungeKutta, tempoAnalise):
    
    avanco = np.zeros((2,1),dtype=np.complex_)
    
    y = 0
    
    horizonte = (horizonteAnalisavel+iteracao)/100
    
    tempo = iteracao/100
    
    tempoInicialAnalise = 0.00
    tempoFinalAnalise = 0.02
    
    controles = np.zeros((2,2), dtype=np.complex_)
    controles = [[0, x[0]], [x[0], 0]]
    
    matrizOrigem = np.zeros((2,1),dtype=np.complex_)
    matrizOrigem = [[onda[0][0]], [onda[1][0]]] 

    matrizDestino = np.zeros((2,1),dtype=np.complex_)
    
    ## Função-Objetivo (Return) = somatorio ||(Matriz_Origem - Matriz_Destino)||^2
    
    ## Avanço temporal
    
    matrizDestino[0][0] = ondaDestino[0][0]*np.exp(-1*complex(0,1)*(1/2*np.pi)*tempo)
    matrizDestino[1][0] = ondaDestino[1][0]*np.exp(-1*complex(0,1)*(3/2*np.pi)*tempo)

    y += (np.linalg.norm(matrizOrigem-matrizDestino))**2

    avanco = rungeKutta(matrizOrigem, fatorRungeKutta, somar(hamiltoniano, controles), tempoFinal = tempoFinalAnalise, tempoInicial = tempoInicialAnalise)

    matrizOrigem = avanco

    tempoInicialAnalise += 0.01
    tempoFinalAnalise += 0.01
    
    for indice in range((iteracao+1), (horizonteAnalisavel+iteracao)):
        tempo = indice/100
        
        matrizDestino[0][0] = ondaDestino[0][0]*np.exp(-1*complex(0,1)*(1/2*np.pi)*tempo)
        matrizDestino[1][0] = ondaDestino[1][0]*np.exp(-1*complex(0,1)*(3/2*np.pi)*tempo)
        
        y += (np.linalg.norm(matrizOrigem-matrizDestino))**2
        
        controles = [[0, x[int(tempoInicialAnalise*100)]], [x[int(tempoInicialAnalise*100)], 0]]

        avanco = rungeKutta(matrizOrigem, fatorRungeKutta, somar(hamiltoniano, controles), tempoFinal = tempoFinalAnalise, tempoInicial = tempoInicialAnalise)

        matrizOrigem = avanco

        tempoInicialAnalise += 0.01
        tempoFinalAnalise += 0.01
    
    return y

In [3]:
import matplotlib.pyplot as plt

In [4]:
import random
import numpy as np
from deap import algorithms, base, creator, tools

hamiltoniano = np.zeros((2,2), dtype=np.complex_) # Hamiltoniano
A = (1/2*np.pi)
B = (3/2*np.pi)
hamiltoniano = [[A, 0], [0, B]]
fatorRungeKutta = 0.01

iteracao = 0
fatorRungeKutta = 0.01
tempoAnalise = 2000
horizonte = 10

ondaOrigem = [[complex(1.0,0.0)],[complex(0.0,0.0)]]
ondaDestino = [[complex(1/np.sqrt(2),0)],[complex(1/np.sqrt(2),0)]]


def evaluate(individual, iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonteAnalisavel, fatorRungeKutta, tempoAnalise):
    teste = funcaoObjetivo(individual, iteracao, ondaDestino, hamiltoniano, ondaOrigem, horizonteAnalisavel, fatorRungeKutta, tempoAnalise)
    return teste.item(),

while iteracao < tempoAnalise:
    # Criando o objeto Fitness
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

    # Criando o objeto Individual
    creator.create("Individual", list, fitness=creator.FitnessMin)

    # Criando o Toolbox
    toolbox = base.Toolbox()

    # Registrando o gerador de números aleatórios
    toolbox.register("attr_float", random.uniform, -5.0, 5.0)

    # Registrando o inicializador de indivíduos
    toolbox.register("individual", tools.initCycle, creator.Individual,
                     (toolbox.attr_float, toolbox.attr_float, toolbox.attr_float,
                     toolbox.attr_float, toolbox.attr_float, toolbox.attr_float,
                     toolbox.attr_float, toolbox.attr_float, toolbox.attr_float,
                     toolbox.attr_float), n=1)

    # Registrando a população
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    # Registrando a função objetivo
    toolbox.register("evaluate", evaluate, iteracao=iteracao, ondaDestino=ondaDestino , hamiltoniano=hamiltoniano , ondaOrigem=ondaOrigem , horizonteAnalisavel=horizonte, fatorRungeKutta=fatorRungeKutta, tempoAnalise=tempoAnalise)

    # Registrando os operadores genéticos
    toolbox.register("mate", tools.cxBlend, alpha=0.5)
    toolbox.register("mutate", tools.mutGaussian, mu=0.0, sigma=0.2, indpb=0.1)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # Criando a população inicial
    pop = toolbox.population(n=10)

    # Avaliando a população inicial
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    # Executando o algoritmo genético
    for g in range(10):
        # Selecionando os indivíduos para cruzamento
        selected = toolbox.select(pop, len(pop))
        # Clonando os indivíduos selecionados
        offspring = [toolbox.clone(ind) for ind in selected]
        # Realizando o cruzamento e a mutação
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.5:
                toolbox.mate(child1, child2)
            toolbox.mutate(child1)
            toolbox.mutate(child2)
            del child1.fitness.values, child2.fitness.values
        # Avaliando os novos indivíduos gerados
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = list(map(toolbox.evaluate, invalid_ind))
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        # Substituindo a população antiga pela nova
        pop[:] = offspring

    # Exibindo o melhor indivíduo encontrado
    best_ind = tools.selBest(pop, 1)[0]
    print("Melhor solução encontrada: f(X) = %.2f" % (best_ind.fitness.values[0]))
    
    controle = np.zeros((2,1), dtype=np.complex_)
    controle = [[0, sum(best_ind.fitness.values)],[sum(best_ind.fitness.values),0]]
    hamiltoniano = somar(hamiltoniano, controle)

    ondaAvancada = np.zeros((2,2), dtype=np.complex_)
    ondaAvancada = rungeKutta(ondaOrigem, fatorRungeKutta, hamiltoniano, tempoFinal = 0.02, tempoInicial = 0)

    ondaOrigem[0][0] = ondaAvancada[0][0]
    ondaOrigem[1][0] = ondaAvancada[1][0]

    A = (1/2*np.pi)
    B = (3/2*np.pi)
    hamiltoniano = [[A, 0], [0, B]]
    
    iteracao += 1


Melhor solução encontrada: f(X) = 5.76


C:\Users\Gabriel\anaconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\Gabriel\anaconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Melhor solução encontrada: f(X) = 5.78
Melhor solução encontrada: f(X) = 5.85
Melhor solução encontrada: f(X) = 5.86
Melhor solução encontrada: f(X) = 5.96
Melhor solução encontrada: f(X) = 6.06
Melhor solução encontrada: f(X) = 6.15
Melhor solução encontrada: f(X) = 6.27
Melhor solução encontrada: f(X) = 6.41
Melhor solução encontrada: f(X) = 6.56
Melhor solução encontrada: f(X) = 6.81
Melhor solução encontrada: f(X) = 7.10
Melhor solução encontrada: f(X) = 7.21
Melhor solução encontrada: f(X) = 7.69
Melhor solução encontrada: f(X) = 7.99
Melhor solução encontrada: f(X) = 8.70
Melhor solução encontrada: f(X) = 8.77
Melhor solução encontrada: f(X) = 9.82
Melhor solução encontrada: f(X) = 10.08
Melhor solução encontrada: f(X) = 10.87
Melhor solução encontrada: f(X) = 11.24
Melhor solução encontrada: f(X) = 13.47
Melhor solução encontrada: f(X) = 14.58
Melhor solução encontrada: f(X) = 15.98
Melhor solução encontrada: f(X) = 18.03
Melhor solução encontrada: f(X) = 20.45
Melhor solução en

Melhor solução encontrada: f(X) = 1.11
Melhor solução encontrada: f(X) = 1.11
Melhor solução encontrada: f(X) = 1.11
Melhor solução encontrada: f(X) = 1.11
Melhor solução encontrada: f(X) = 1.10
Melhor solução encontrada: f(X) = 1.10
Melhor solução encontrada: f(X) = 1.11
Melhor solução encontrada: f(X) = 1.10
Melhor solução encontrada: f(X) = 1.11
Melhor solução encontrada: f(X) = 1.11
Melhor solução encontrada: f(X) = 1.11
Melhor solução encontrada: f(X) = 1.12
Melhor solução encontrada: f(X) = 1.13
Melhor solução encontrada: f(X) = 1.12
Melhor solução encontrada: f(X) = 1.13
Melhor solução encontrada: f(X) = 1.13
Melhor solução encontrada: f(X) = 1.15
Melhor solução encontrada: f(X) = 1.15
Melhor solução encontrada: f(X) = 1.16
Melhor solução encontrada: f(X) = 1.18
Melhor solução encontrada: f(X) = 1.18
Melhor solução encontrada: f(X) = 1.20
Melhor solução encontrada: f(X) = 1.22
Melhor solução encontrada: f(X) = 1.22
Melhor solução encontrada: f(X) = 1.24
Melhor solução encontrada

Melhor solução encontrada: f(X) = 1.49
Melhor solução encontrada: f(X) = 1.50
Melhor solução encontrada: f(X) = 1.50
Melhor solução encontrada: f(X) = 1.49
Melhor solução encontrada: f(X) = 1.49
Melhor solução encontrada: f(X) = 1.50
Melhor solução encontrada: f(X) = 1.50
Melhor solução encontrada: f(X) = 1.50
Melhor solução encontrada: f(X) = 1.51
Melhor solução encontrada: f(X) = 1.53
Melhor solução encontrada: f(X) = 1.54
Melhor solução encontrada: f(X) = 1.53
Melhor solução encontrada: f(X) = 1.55
Melhor solução encontrada: f(X) = 1.57
Melhor solução encontrada: f(X) = 1.57
Melhor solução encontrada: f(X) = 1.60
Melhor solução encontrada: f(X) = 1.62
Melhor solução encontrada: f(X) = 1.64
Melhor solução encontrada: f(X) = 1.65
Melhor solução encontrada: f(X) = 1.72
Melhor solução encontrada: f(X) = 1.71
Melhor solução encontrada: f(X) = 1.70
Melhor solução encontrada: f(X) = 1.76
Melhor solução encontrada: f(X) = 1.80
Melhor solução encontrada: f(X) = 1.85
Melhor solução encontrada

...

Melhor solução encontrada: f(X) = 2.93
Melhor solução encontrada: f(X) = 2.96
Melhor solução encontrada: f(X) = 2.98
Melhor solução encontrada: f(X) = 3.04
Melhor solução encontrada: f(X) = 3.08
Melhor solução encontrada: f(X) = 3.12
Melhor solução encontrada: f(X) = 3.16
Melhor solução encontrada: f(X) = 3.24
Melhor solução encontrada: f(X) = 3.29
Melhor solução encontrada: f(X) = 3.43
Melhor solução encontrada: f(X) = 3.45
Melhor solução encontrada: f(X) = 3.55
Melhor solução encontrada: f(X) = 3.61
Melhor solução encontrada: f(X) = 3.84
Melhor solução encontrada: f(X) = 4.05
Melhor solução encontrada: f(X) = 4.13
Melhor solução encontrada: f(X) = 4.39
Melhor solução encontrada: f(X) = 4.61
Melhor solução encontrada: f(X) = 4.91
Melhor solução encontrada: f(X) = 5.28
Melhor solução encontrada: f(X) = 5.09
Melhor solução encontrada: f(X) = 5.68
Melhor solução encontrada: f(X) = 6.24
Melhor solução encontrada: f(X) = 6.65
Melhor solução encontrada: f(X) = 6.72
Melhor solução encontrada

Melhor solução encontrada: f(X) = 1.40
Melhor solução encontrada: f(X) = 1.56
Melhor solução encontrada: f(X) = 1.59
Melhor solução encontrada: f(X) = 1.74
Melhor solução encontrada: f(X) = 1.79
Melhor solução encontrada: f(X) = 1.95
Melhor solução encontrada: f(X) = 1.92
Melhor solução encontrada: f(X) = 1.98
Melhor solução encontrada: f(X) = 2.21
Melhor solução encontrada: f(X) = 2.39
Melhor solução encontrada: f(X) = 2.42
Melhor solução encontrada: f(X) = 2.60
Melhor solução encontrada: f(X) = 2.76
Melhor solução encontrada: f(X) = 2.92
Melhor solução encontrada: f(X) = 2.79
Melhor solução encontrada: f(X) = 3.12
Melhor solução encontrada: f(X) = 3.65
Melhor solução encontrada: f(X) = 3.59
Melhor solução encontrada: f(X) = 3.62
Melhor solução encontrada: f(X) = 4.18
Melhor solução encontrada: f(X) = 4.49
Melhor solução encontrada: f(X) = 5.02
Melhor solução encontrada: f(X) = 4.98
Melhor solução encontrada: f(X) = 5.74
Melhor solução encontrada: f(X) = 6.39
Melhor solução encontrada

Melhor solução encontrada: f(X) = 2.16
Melhor solução encontrada: f(X) = 1.54
Melhor solução encontrada: f(X) = 1.53
Melhor solução encontrada: f(X) = 1.31
Melhor solução encontrada: f(X) = 1.07
Melhor solução encontrada: f(X) = 1.45
Melhor solução encontrada: f(X) = 1.34
Melhor solução encontrada: f(X) = 1.08
Melhor solução encontrada: f(X) = 1.22
Melhor solução encontrada: f(X) = 1.08
Melhor solução encontrada: f(X) = 0.90
Melhor solução encontrada: f(X) = 1.06
Melhor solução encontrada: f(X) = 0.90
Melhor solução encontrada: f(X) = 1.05
Melhor solução encontrada: f(X) = 0.82
Melhor solução encontrada: f(X) = 0.94
Melhor solução encontrada: f(X) = 0.97
Melhor solução encontrada: f(X) = 0.84
Melhor solução encontrada: f(X) = 0.75
Melhor solução encontrada: f(X) = 0.86
Melhor solução encontrada: f(X) = 0.66
Melhor solução encontrada: f(X) = 0.69
Melhor solução encontrada: f(X) = 0.67
Melhor solução encontrada: f(X) = 0.68
Melhor solução encontrada: f(X) = 0.69
Melhor solução encontrada

Melhor solução encontrada: f(X) = 6.89
Melhor solução encontrada: f(X) = 6.64
Melhor solução encontrada: f(X) = 6.35
Melhor solução encontrada: f(X) = 6.18
Melhor solução encontrada: f(X) = 6.18
Melhor solução encontrada: f(X) = 6.02
Melhor solução encontrada: f(X) = 5.97
Melhor solução encontrada: f(X) = 5.95
Melhor solução encontrada: f(X) = 5.91
Melhor solução encontrada: f(X) = 5.91
Melhor solução encontrada: f(X) = 5.91
Melhor solução encontrada: f(X) = 5.93
Melhor solução encontrada: f(X) = 6.00
Melhor solução encontrada: f(X) = 6.04
Melhor solução encontrada: f(X) = 6.16
Melhor solução encontrada: f(X) = 6.23
Melhor solução encontrada: f(X) = 6.31
Melhor solução encontrada: f(X) = 6.51
Melhor solução encontrada: f(X) = 6.60
Melhor solução encontrada: f(X) = 7.09
Melhor solução encontrada: f(X) = 7.19
Melhor solução encontrada: f(X) = 7.67
Melhor solução encontrada: f(X) = 7.65
Melhor solução encontrada: f(X) = 8.23
Melhor solução encontrada: f(X) = 8.95
Melhor solução encontrada

Melhor solução encontrada: f(X) = 24.89
Melhor solução encontrada: f(X) = 27.95
Melhor solução encontrada: f(X) = 31.64
Melhor solução encontrada: f(X) = 33.30
Melhor solução encontrada: f(X) = 34.68
Melhor solução encontrada: f(X) = 33.47
Melhor solução encontrada: f(X) = 31.24
Melhor solução encontrada: f(X) = 27.66
Melhor solução encontrada: f(X) = 24.14
Melhor solução encontrada: f(X) = 20.81
Melhor solução encontrada: f(X) = 17.82
Melhor solução encontrada: f(X) = 14.00
Melhor solução encontrada: f(X) = 13.55
Melhor solução encontrada: f(X) = 11.75
Melhor solução encontrada: f(X) = 10.77
Melhor solução encontrada: f(X) = 9.39
Melhor solução encontrada: f(X) = 9.32
Melhor solução encontrada: f(X) = 7.76
Melhor solução encontrada: f(X) = 7.43
Melhor solução encontrada: f(X) = 7.11
Melhor solução encontrada: f(X) = 7.12
Melhor solução encontrada: f(X) = 6.52
Melhor solução encontrada: f(X) = 6.54
Melhor solução encontrada: f(X) = 6.36
Melhor solução encontrada: f(X) = 6.03
Melhor sol